# Mongolia Lineaments #

Using [geemap](https://geemap.org/) to analyze lineaments generated from Landsat 7 and SRTM in Mongolia. Edges were detected by using [Canny Edge Detector](https://developers.google.com/earth-engine/apidocs/ee-algorithms-cannyedgedetector?hl=en) and further analyzed through [Hough Transform](https://developers.google.com/earth-engine/apidocs/ee-algorithms-houghtransform?hl=en).

Layer tab in top right can turn on/off layers as well as change opacity.

Data Sources -
* [SRTM](https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001?hl=en) (green)
* [Landsat 7](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1?hl=en) (red)

In [21]:
# Import earth engine and geemap

import ee
import geemap
geemap.ee_initialize()

#############Import Shapefiles###########
mongoShape = ee.FeatureCollection("users/pirrie/MongoliaAssets/MongoliaOutline")
potSuture = ee.FeatureCollection("users/pirrie/MongoliaAssets/PotentialSuture")
mongoGeom = mongoShape.geometry()

Map = geemap.Map(center=[46.935,103.376], zoom=5)

# Add Mongolia Shape, Suture
Map.addLayer(mongoShape, {'color': '000000'}, 'Mongolia', True, 0.3)
Map.addLayer(potSuture, {'color': 'FFFF00'}, 'Suture (Zorin)', True, 0.5)

# Load Landsat 7 data, filter by date and bounds.
collection = ee.ImageCollection("LANDSAT/LE07/C01/T1") \
  .filterDate('2019-01-01', '2021-08-18') \
  .filter(ee.Filter.calendarRange(6, 9, 'month'))

# Also filter the collection by the IMAGE_QUALITY property.
filtered = collection \
  .filterMetadata('IMAGE_QUALITY', 'equals', 9)

# Create two composites to check the effect of filtering by IMAGE_QUALITY.
goodComposite = ee.Algorithms.Landsat.simpleComposite(filtered, 75, 3)

# Add PC band
Panchromatic = goodComposite.select(['B8']) # \
  # .clip(mongoShape) \
  # .select(['B8'])

Map.addLayer(Panchromatic,
             {'gain': 1.5},
             'Panchromatic',
             False)

# Add SRTM 30m (https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001?hl=en)
DEM = ee.Image('NASA/NASADEM_HGT/001').select('elevation')


#############Edge Detection###########

# Detect edges in the panchromatic composite. Variables are (image, threshold, sigma)
cannyLS = ee.Algorithms.CannyEdgeDetector(Panchromatic, 10, 1.5)
cannyDEM = ee.Algorithms.CannyEdgeDetector(DEM, 10, 1.5)


# Mask the image with itself to get rid of areas with no edges.
cannyLS = cannyLS \
  .updateMask(cannyLS)
cannyDEM = cannyDEM \
  .updateMask(cannyDEM)

Map.addLayer(cannyLS, {'min': 0, 'max': 1, 'palette': '0620DB'}, 'CE Pan LS', False)
Map.addLayer(cannyDEM, {'min': 0, 'max': 1, 'palette': '8F1713'}, 'CE DEM', False)


#############Hough Transformation###########

# Perform Hough transform of the Canny result and display.
houghLS = ee.Algorithms.HoughTransform(cannyLS, 256, 64, 300, True)
houghDEM = ee.Algorithms.HoughTransform(cannyDEM, 256, 64, 300, True)

Map.addLayer(houghLS, {'palette': '1293DB'}, 'Hough LS', False, 0.5)
Map.addLayer(houghDEM, {'palette': 'DB3832'}, 'Hough DEM', False, 0.5)

bufferSize = 250
houghLSbuffer = houghLS.focal_max(bufferSize, 'square', 'meters')
houghDEMbuffer = houghDEM.focal_max(bufferSize, 'square', 'meters')

Map.addLayer(houghLSbuffer, {'palette': '1293DB'}, 'Hough LS w/Buffer', True, 0.5)
Map.addLayer(houghDEMbuffer, {'palette': 'DB3832'}, 'Hough DEM w/Buffer', True, 0.5)

Map

Map(center=[46.935, 103.376], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

Red line is proposed suture after Zorin (1999)